In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from annoy import AnnoyIndex
import random

In [2]:
debug=False
doTest=False
nrBucketsToDo=1
bucketSize=20

In [3]:
inputFolderName="/Users/luizaadelinaciucu/Work/ATLAS/TrackML/input/ttbar_mu200-generic"
eventNumber="000000099"
inputFileName_hits_recon=inputFolderName+"/event"+eventNumber+"-hits.csv"
inputFileName_hits_truth=inputFolderName+"/event"+eventNumber+"-truth.csv"

In [4]:
outputFolderName="/Users/luizaadelinaciucu/Work/ATLAS/TrackMLtest/output3"

fileNameNNInput=outputFolderName+"/NN_1_data_Input_"+eventNumber+".npy"
fileNameNNOutput=outputFolderName+"/NN_1_data_Output_"+eventNumber+".npy"

fileNameNNInputTrain=outputFolderName+"/NN_2_data_Input_Train_"+eventNumber+".npy"
fileNameNNOutputTrain=outputFolderName+"/NN_2_data_Output_Train_"+eventNumber+".npy"

fileNameNNInputTest=outputFolderName+"/NN_2_data_Input_Test_"+eventNumber+".npy"
fileNameNNOutputTest=outputFolderName+"/NN_2_data_Output_Test_"+eventNumber+".npy"

In [5]:
df_hits_recon=pd.read_csv(inputFileName_hits_recon)
df_hits_truth=pd.read_csv(inputFileName_hits_truth)
df_hits=pd.concat([df_hits_recon,df_hits_truth],axis=1,sort=False)
df_hits.head()

,hit_id,x,y,z,volume_id,layer_id,module_id,hit_id,particle_id,tx,ty,tz,tpx,tpy,tpz
0,0,53.6895,8.81461,-1502.5,7,2,1,0,18014467228958720,53.6668,8.82130,-1502.5,0.973584,0.159988,-28.120100
1,1,70.7608,7.70057,-1502.5,7,2,1,1,22528787094700032,70.7375,7.71636,-1502.5,0.213369,0.023315,-4.386220
2,2,80.2306,11.40500,-1502.5,7,2,1,2,63052593806442496,80.2098,11.40200,-1502.5,0.050232,0.007147,-0.985516
3,3,63.7141,4.03639,-1502.5,7,2,1,3,103588701304520704,63.7169,4.02437,-1502.5,0.196107,0.012337,-4.797500
4,4,63.8302,3.99537,-1502.5,7,2,1,4,117101080734597120,63.8096,4.00231,-1502.5,0.049758,0.003124,-1.159650


Creating buckets

In [6]:
df_hits["particle_id"][3]

103588701304520704

In [7]:
len(df_hits)

116893

For certain hits produce a bucket

In [8]:
def buildAnnoyIndex(nparray_position,metric="angular",ntrees=10,debug=False): 
    numberDimension=nparray_position.shape[1] # 3 (x,y,z)
    if debug:
        print("numberDimension",numberDimension)
    index=AnnoyIndex(numberDimension,metric)
    if debug:
        print("type(index)",type(index))
        print("enumerate data")
    # add each hit to the index
    for i,position in enumerate(nparray_position):
        if debug:
            print("i",i,"position",position)
        index.add_item(i,position)
    # done for loop over hits
    # build the index with 10 trees
    index.build(ntrees) # 10 trees
    return index
# done function


In [9]:
nparray_position=df_hits[["x","y","z"]].values
print("nparray_position",nparray_position)
index=buildAnnoyIndex(nparray_position,metric="angular",ntrees=10,debug=False)

nparray_position [[   53.6895      8.81461 -1502.5    ]
 [   70.7608      7.70057 -1502.5    ]
 [   80.2306     11.405   -1502.5    ]
 ...
 [ -937.832      95.8561   2952.5    ]
 [ -901.023      18.392    2952.5    ]
 [ -876.783      72.4166   2952.5    ]]


In [10]:
list_nparray_input=[]
list_nparray_output=[]
nparray_volume_id=df_hits["volume_id"].values
nparray_layer_id=df_hits["layer_id"].values
counter=0
for i in range(len(df_hits)):
    if doTest==True and counter>(nrBucketsToDo-1):
        continue
    if (nparray_volume_id[i]==8 and nparray_layer_id[i]==2)==False:
        continue
    counter+=1
    if debug==True:
        print(i)
    # using annoy to find the 20 nearest neighboring hits by angle to this hit
    list_index=index.get_nns_by_item(i,bucketSize)
    if debug:
        print("list_index",list_index)
    # create bucket
    df_bucket=df_hits.iloc[list_index]
    # create NN input for this one bucket
    nparray_input=df_bucket[["x","y","z"]].values.flatten()
    if debug:
        print("nparray_input",nparray_input,"shape",nparray_input.shape,"type",type(nparray_input))       
    # create NN output for this one bucket
    # identify particle with the largest number of hits in this bucket
    nparray_particle_id=df_bucket["particle_id"].values
    if debug:
        print("nparray_particle_id",nparray_particle_id,"shape",nparray_particle_id.shape,"type",type(nparray_particle_id))
    dict_particle_id_counter={}
    for particle_id in nparray_particle_id:
        if particle_id not in dict_particle_id_counter:
            dict_particle_id_counter[particle_id]=1
        else:
            dict_particle_id_counter[particle_id]+=1
    if debug:
        print("dict_particle_id_counter",dict_particle_id_counter)
    # find the maximum value of the counters
    particle_id_with_max_hits=0
    counter_max=0
    for particle_id in dict_particle_id_counter:
        counter=dict_particle_id_counter[particle_id]
        if counter>counter_max:
            counter_max=counter
            particle_id_with_max_hits=particle_id
    if debug:
        print("particle_id_with_max_hits",particle_id_with_max_hits)
        
    # create list output
    list_output=[]
    for particle_id in nparray_particle_id:
        if particle_id==particle_id_with_max_hits:
            list_output.append(1)
        else:
            list_output.append(-1)
    if debug:        
        print("list_output",list_output)
    nparray_output=np.array(list_output)
    if debug:
        print("nparray_output",nparray_output)
            
    
    if debug and doTest:
        plt.plot(df_bucket.x,df_bucket.y,"o",color="red")
        plt.plot(0,0,"r+")
        plt.title(str(i))
        plt.show()
        # plt.savefig(outputFolderName+"/bucket_"+str(i)+".pdf")
        
    list_nparray_input.append(nparray_input)
    list_nparray_output.append(nparray_output)
       
# done for loop
nparray_input_all=np.array(list_nparray_input)
nparray_output_all=np.array(list_nparray_output)
print("nparray_input_all",nparray_input_all,"shape",nparray_input_all.shape,"type",type(nparray_input_all))
print("nparray_output_all",nparray_output_all,"shape",nparray_output_all.shape,"type",type(nparray_output_all))

nparray_input_all [[-2.91408e+01 -1.34121e+01 -4.53283e+02 ... -8.72973e+01 -3.56456e+01
  -1.30200e+03]
 [-3.17481e+01 -5.95475e+00 -4.21849e+02 ... -4.67051e+01 -6.50951e+00
  -6.02500e+02]
 [-3.08955e+01 -8.39350e+00 -4.47112e+02 ... -5.31951e+01 -1.57277e+01
  -8.18000e+02]
 ...
 [-3.31662e+01 -3.98620e-01  4.46810e+02 ... -7.94153e+01  2.56644e-01
   1.09750e+03]
 [-3.12528e+01  4.19333e+00  4.50675e+02 ... -6.78425e+01  1.15825e+01
   9.62000e+02]
 [-3.07637e+01  1.28645e+01  4.24059e+02 ... -5.61753e+01  2.31333e+01
   8.18000e+02]] shape (10653, 60) type <class 'numpy.ndarray'>
nparray_output_all [[-1 -1 -1 ... -1 -1  1]
 [-1 -1  1 ...  1 -1 -1]
 [ 1  1  1 ... -1 -1 -1]
 ...
 [ 1  1  1 ... -1 -1 -1]
 [ 1  1  1 ... -1 -1 -1]
 [-1 -1 -1 ... -1 -1 -1]] shape (10653, 20) type <class 'numpy.ndarray'>


In [11]:
# we will want later to split in half for train and test
# so keep only an even number
# if an odd number of buckets, remove the last one
nrBuckets=nparray_input_all.shape[0]
if nrBuckets%2==1:
    # odd number of buckets
    # rewrite the input as to remove the last element
    nparray_input_all=nparray_input_all[0:-1]
    # rewrite the output as to remove the last element
    nparray_output_all=nparray_output_all[0:-1]
else:
    # if even, do nothing
    pass
# done if
print("nparray_input_all",nparray_input_all,"shape",nparray_input_all.shape,"type",type(nparray_input_all))
print("nparray_output_all",nparray_output_all,"shape",nparray_output_all.shape,"type",type(nparray_output_all))

nparray_input_all [[-2.91408e+01 -1.34121e+01 -4.53283e+02 ... -8.72973e+01 -3.56456e+01
  -1.30200e+03]
 [-3.17481e+01 -5.95475e+00 -4.21849e+02 ... -4.67051e+01 -6.50951e+00
  -6.02500e+02]
 [-3.08955e+01 -8.39350e+00 -4.47112e+02 ... -5.31951e+01 -1.57277e+01
  -8.18000e+02]
 ...
 [-3.12786e+01  3.73645e+00  4.79993e+02 ... -5.98624e+01  7.09361e+00
   9.57500e+02]
 [-3.31662e+01 -3.98620e-01  4.46810e+02 ... -7.94153e+01  2.56644e-01
   1.09750e+03]
 [-3.12528e+01  4.19333e+00  4.50675e+02 ... -6.78425e+01  1.15825e+01
   9.62000e+02]] shape (10652, 60) type <class 'numpy.ndarray'>
nparray_output_all [[-1 -1 -1 ... -1 -1  1]
 [-1 -1  1 ...  1 -1 -1]
 [ 1  1  1 ... -1 -1 -1]
 ...
 [ 1  1  1 ... -1 -1 -1]
 [ 1  1  1 ... -1 -1 -1]
 [ 1  1  1 ... -1 -1 -1]] shape (10652, 20) type <class 'numpy.ndarray'>


In [17]:
# reshape for the input to have an extra dimension of size one, needed by Keras
nparray_input_all=nparray_input_all.reshape(nparray_input_all.shape[0],nparray_input_all.shape[1],1)
print("nparray_input_all",nparray_input_all,"shape",nparray_input_all.shape,"type",type(nparray_input_all))

nparray_input_all [[[-2.91408e+01]
  [-1.34121e+01]
  [-4.53283e+02]
  ...
  [-8.72973e+01]
  [-3.56456e+01]
  [-1.30200e+03]]

 [[-3.17481e+01]
  [-5.95475e+00]
  [-4.21849e+02]
  ...
  [-4.67051e+01]
  [-6.50951e+00]
  [-6.02500e+02]]

 [[-3.08955e+01]
  [-8.39350e+00]
  [-4.47112e+02]
  ...
  [-5.31951e+01]
  [-1.57277e+01]
  [-8.18000e+02]]

 ...

 [[-3.12786e+01]
  [ 3.73645e+00]
  [ 4.79993e+02]
  ...
  [-5.98624e+01]
  [ 7.09361e+00]
  [ 9.57500e+02]]

 [[-3.31662e+01]
  [-3.98620e-01]
  [ 4.46810e+02]
  ...
  [-7.94153e+01]
  [ 2.56644e-01]
  [ 1.09750e+03]]

 [[-3.12528e+01]
  [ 4.19333e+00]
  [ 4.50675e+02]
  ...
  [-6.78425e+01]
  [ 1.15825e+01]
  [ 9.62000e+02]]] shape (10652, 60, 1) type <class 'numpy.ndarray'>


In [15]:
# save them to files
np.save(fileNameNNInput,nparray_input_all)
np.save(fileNameNNOutput,nparray_output_all)

In [16]:
# split each of input and output by half in train and test
nrBucket=nparray_input_all.shape[0] # is now an even number
# create list of even indices to be used for train
list_index_Train=[i for i in range(nrBucket) if i%2==0]
# create list of odd indices to be used for train
list_index_Test=[i for i in range(nrBucket) if i%2==1]

# split input in Train and Test by giving as index a list of indices
nparray_Input_Train=nparray_input_all[list_index_Train]
nparray_Input_Test=nparray_input_all[list_index_Test]
nparray_Output_Train=nparray_output_all[list_index_Train]
nparray_Output_Test=nparray_output_all[list_index_Test]

# save them to files
np.save(fileNameNNInputTrain,nparray_Input_Train)
np.save(fileNameNNInputTest,nparray_Input_Test)
np.save(fileNameNNOutputTrain,nparray_Output_Train)
np.save(fileNameNNOutputTest,nparray_Output_Test)